# Importação de Bibliotecas

In [ ]:
import numpy as np
import torch
import numpy as np
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import matplotlib.pyplot as plt

# Definição de variáveis

In [ ]:
in_features = 5
out_features = 3

# Definição do modelo

In [ ]:
class CreditNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(in_features, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 8),
        nn.ReLU(),
        nn.Linear(8, out_features),
    )

  def forward(self, x):
    return self.layers(x)

# Preparando os dados de entrada

In [ ]:
class CreditDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
baseUrl = "https://raw.githubusercontent.com/Yuri-Chaves/NeuralNetwork/refs/heads/main/Dataset"
dfTrain = pd.read_csv(f"{baseUrl}/clientes_credito_train.csv")
dfTest = pd.read_csv(f"{baseUrl}/clientes_credito_test.csv")

# Selecionar as colunas relevantes
XTrain = dfTrain[["idade", "renda_mensal", "historico_pagamento", "dividas_ativas", "score_credito"]].values
YTrain = dfTrain["grupo_credito"].map({"green": 0, "yellow": 1, "red": 2}).values

XTest = dfTest[["idade", "renda_mensal", "historico_pagamento", "dividas_ativas", "score_credito"]].values
YTest = dfTest["grupo_credito"].map({"green": 0, "yellow": 1, "red": 2}).values

In [ ]:
trainDataset = CreditDataset(XTrain, YTrain)
testDataset = CreditDataset(XTest, YTest)

trainDataLoader = DataLoader(trainDataset, batch_size=32, shuffle=True)
testDataLoader = DataLoader(testDataset, batch_size=32, shuffle=False)

# Hiperparâmetros de Otimização

In [ ]:
# Validando ambiente de execução
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Rodando na {device}")

In [ ]:
model = CreditNetwork().to(device)

In [ ]:
# Função de perda (loss function)
# Erro quadrático médio (Mean Squared Error) MSELoss Usado para regressão
# Perda de entropia cruzada (CrossEntropyLoss) Calcula a perda de entropia cruzada entre logits de entrada e alvo
lossFunc = nn.CrossEntropyLoss()
# Gradiente Descendente Estocástico
# SGD = Stochastic Gradient Descent
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# taxa de aprendizado lr = learning rate

# Função de treinamento

In [ ]:
def train(model, dataloader, lossFunc, optimizer):
  model.train()
  cumLoss = 0.0
  for X, y in dataloader:
    X = X.float().to(device)
    y = y.float().to(device)

    pred = model(X)
    loss = lossFunc(pred, y)

    # zera os gradientes acumulados
    optimizer.zero_grad()
    # computa os gradientes
    loss.backward()
    # anda, de fato, na direção que reduz o erro local
    optimizer.step()

    # loss é um tensor; item pra obter o float
    cumLoss += loss.item()

  return cumLoss / len(dataloader)

# Função de teste

In [ ]:
def test(model, dataloader, lossFunc):
  model.eval()

  cumLoss = 0.0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.float().to(device)
      y = y.float().to(device)

      pred = model(X)
      loss = lossFunc(pred, y)
      cumLoss += loss.item()

  return cumLoss / len(dataloader)

# Função para visualização do treinamento

In [ ]:
def plotComparison(model, XTest, YTest, device):
    fig, ax = plt.subplots(figsize=(10, 6))

    ax.grid(True, which='both')
    ax.spines['left'].set_position('zero')
    ax.spines['right'].set_color('none')
    ax.spines['bottom'].set_position('zero')
    ax.spines['top'].set_color('none')

    # Converter X_test para tensor e fazer previsões
    XTensor = torch.tensor(XTest, dtype=torch.float32).to(device)
    model.eval()
    with torch.no_grad():
        yPred = model(XTensor)  # Obter logits ou probabilidades

    # Converter para classes (0, 1, 2)
    y_pred_classes = torch.argmax(yPred, dim=1).cpu().numpy()

    # Definir cores para cada grupo de crédito
    colors = {0: "green", 1: "orange", 2: "red"}

    # Criar scatter plot (Gráfico de dispersão)
    for i in range(3):  # Para cada classe (0, 1, 2)
        indices = np.where(YTest == i)[0]  # Índices dos exemplos dessa classe
        ax.scatter(indices, YTest[indices], color=colors[i], label=f"Real {i}", marker="o")
        ax.scatter(indices, y_pred_classes[indices], color=colors[i], marker="x", label=f"Pred {i}")

    ax.set_xlabel("Amostras")
    ax.set_ylabel("Grupo de Crédito")
    plt.legend()
    plt.show()

# Treinamento

In [ ]:
epochs = 201
for t in range(epochs):
  trainLoss = train(model, trainDataLoader, lossFunc, optimizer)
  if t % 50 == 0:
    print(f"Epoch: {t}; Train Loss: {trainLoss}")
    plotComparison(model, XTest, YTest, device)

# Teste

In [ ]:
testLoss = test(model, testDataLoader, lossFunc)
print(f"Test Loss: {testLoss}")
plotComparison(model, XTest, YTest, device)